## python-dotenv

In [2]:
from dotenv import load_dotenv, find_dotenv
import os


load_dotenv(find_dotenv(), override=True)

True

## ChatModels

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192", temperature=0.1)

response = llm.invoke("What is the capital of France?")
print(response.content)

The capital of France is Paris.


In [6]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the capital of France?")
]

response = llm.invoke(messages)
print(response.content)

The capital of France is Paris!


## Caching LLM Response
#### In-memory Cache

In [8]:
%%time
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())
prompt = "What is the capital of France?"
response = llm.invoke(prompt)
print(response.content)

The capital of France is Paris.
CPU times: total: 15.6 ms
Wall time: 299 ms


In [9]:
%%time
response = llm.invoke(prompt)
print(response.content)

The capital of France is Paris.
CPU times: total: 0 ns
Wall time: 743 μs


## LLM Streaming

In [13]:
prompt = "Write a poem about haiku?"

for chunk in llm.stream(prompt):
    print(chunk.content, end="", flush=True)

Haiku's gentle art
Syllables in perfect part
Nature's fleeting heart

In three lines, a tale is told
Of seasons, sun, and snow cold
Moments to behold

The traditional form
Five-seven-five, a wistful norm
Concise, yet profound

A glimpse of life's brief span
A haiku captures the plan
Of nature's subtle hand

In haiku's quiet space
The world's beauty finds its place
A moment's peaceful face

The art of haiku's skill
To capture life's fleeting will
In few, well-chosen words still

A haiku's beauty lies
In its simplicity's guise
A reflection of the skies

## Prompt Templates

In [15]:
from langchain.prompts import PromptTemplate

template = """You are a football expert. write few lines about {team} football team in {language}."""

prompt_template = PromptTemplate(template=template)
prompt = prompt_template.format(team="AC Milan", language="English")

response = llm.invoke(prompt)
print(response.content)

AC Milan, one of the most successful and storied clubs in Italian football history! Founded in 1899, the Rossoneri (Red and Blacks) have won an impressive 18 Serie A titles, 5 European Cups, and 2 Intercontinental Cups. With a rich legacy of talented players, including the likes of Paolo Maldini, Franco Baresi, and Andriy Shevchenko, AC Milan has consistently been a force to be reckoned with in European football. Under the guidance of legendary coaches like Arrigo Sacchi and Carlo Ancelotti, the team has developed a strong identity built on defensive solidity, attacking flair, and a never-say-die spirit. With a passionate fan base and a state-of-the-art stadium, the San Siro, AC Milan remains a beloved and respected institution in the world of football.


## Chat Prompt Templates

In [18]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage   

system_message = SystemMessage(content="You respond only in JSON format.")
human_message = HumanMessagePromptTemplate.from_template(
    "List the top {n} tourist attractions in {country}."
)
chat_prompt = ChatPromptTemplate.from_messages( 
    [system_message, human_message] 
)
prompt = chat_prompt.format_prompt(country="Finland",n=2)
response = llm.invoke(prompt)   
print(response.content)

{
"topTouristAttractions": [
{
"name": "Suomenlinna Sea Fortress",
"description": "A UNESCO World Heritage Site located in Helsinki, Finland, known for its historic military fortress and stunning harbor views."
},
{
"name": "Lapland",
"description": "A vast and remote region in northern Finland, known for its breathtaking natural beauty, including the Northern Lights, snow-covered landscapes, and traditional Sami culture."
}
]
}


## Simple Chains

In [24]:
from langchain.chains import LLMChain

llm= ChatGroq(model="meta-llama/llama-4-maverick-17b-128e-instruct", temperature=0.1)

template = """You are a football expert. write few lines about {team} football team in {language}."""

prompt_template = PromptTemplate(template=template)

chain = LLMChain(llm=llm, prompt=prompt_template, verbose=True)

response = chain.invoke({"team": "AC Milan", "language": "English"})

print(response['text'])




> Entering new LLMChain chain...
Prompt after formatting:
You are a football expert. write few lines about AC Milan football team in English.

> Finished chain.
AC Milan is one of the most successful and storied football clubs in the world. Based in Milan, Italy, the team has a rich history of competing at the highest level, having won numerous domestic and international titles, including seven European Cup/UEFA Champions League trophies. With a passionate fan base and a legacy of producing talented players, AC Milan has established itself as a force to be reckoned with in the football world. The team's iconic red and black stripes are synonymous with excellence, and their home stadium, the San Siro, is one of the most famous and intimidating venues in the sport.


## Sequential Chains

In [29]:
from langchain.chains import SimpleSequentialChain

llm1 = ChatGroq(model="qwen/qwen3-32b", temperature=0.1)
llm2 = ChatGroq(model="meta-llama/llama-4-maverick-17b-128e-instruct", temperature=0.1)

template1 = """You are a python expert. write a function that implements a {concept} in python."""
prompt_template1 = PromptTemplate(template=template1)

template2 = """Given the following python function, write a test case for it:\n\n{function}"""
prompt_template2 = PromptTemplate(template=template2)

chain1 = LLMChain(llm=llm1, prompt=prompt_template1, verbose=True)
chain2 = LLMChain(llm=llm2, prompt=prompt_template2, verbose=True)

sequential_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

response = sequential_chain.invoke("decorator")
print(response['output'])



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a python expert. write a function that implements a decorator in python.

> Finished chain.
<think>
Okay, I need to write a Python function that implements a decorator. Let me think about how decorators work in Python. Decorators are functions that wrap another function to extend its behavior without modifying its code. They usually take a function as an argument and return a new function.

So, the basic structure of a decorator is something like:

def decorator(func):
    def wrapper(*args, **kwargs):
        # do something before
        result = func(*args, **kwargs)
        # do something after
        return result
    return wrapper

Then, you can apply it using the @ syntax above a function.

But the user wants me to write a function that implements a decorator. Maybe they want an example of a specific decorator? Since they didn't specify, maybe I should create a g